In [1]:
import gradio as gr
from py_scripts.groq_analysis import *
from py_scripts.gradio_html import *

set_env("GROQ_API_KEY")

In [2]:
with open("assets/js/custom.js", "r", encoding="utf-8") as f:
    js_code = f.read()

In [3]:
recipe_html_block = gr.HTML("⚠️ 請先搜尋想要的食譜。")

with gr.Blocks(css_paths="assets/css/style.css", js=js_code) as demo:
    gr.Markdown("# 🍳 食譜搜尋與問答系統")
    gr.Markdown("歡迎使用本系統，請透過下方各功能分頁搜尋、分析及詢問您喜愛的食譜。")

    recipes_state = gr.State([])    # recipe_list

    with gr.TabItem("搜尋網址"):
        with gr.Column():
            gr.Markdown("### 🔍 食譜搜尋條件")
            with gr.Row():
                text_input = gr.Textbox(label="輸入你要搜尋的食譜名稱", value="餅乾", placeholder="例如：餅乾")
                search_number = gr.Number(label="搜尋食譜數量", value=5, precision=0)

            gr.Markdown("### ⚙️ 進階搜尋條件 (可選)")
            with gr.Group():
                with gr.Row():
                    with gr.Group():
                        like_on = gr.Checkbox(label="啟用讚數", value=False)
                        with gr.Row():
                            like_min = gr.Number(label="讚數最小值", value=0)
                            like_max = gr.Number(label="讚數最大值", value=10000)
                    with gr.Group():
                        together_on = gr.Checkbox(label="啟用一起做數", value=False)
                        with gr.Row():
                            together_min = gr.Number(label="一起做數最小值", value=0)
                            together_max = gr.Number(label="一起做數最大值", value=10000)
                    with gr.Group():
                        comment_on = gr.Checkbox(label="啟用留言數", value=False)
                        with gr.Row():
                            comment_min = gr.Number(label="留言數最小值", value=0)
                            comment_max = gr.Number(label="留言數最大值", value=10000)
                    with gr.Group():
                        view_on = gr.Checkbox(label="啟用瀏覽數", value=False)
                        with gr.Row():
                            view_min = gr.Number(label="瀏覽數最小值", value=0)
                            view_max = gr.Number(label="瀏覽數最大值", value=100000)

            gr.Markdown("### 作者進階搜尋條件")
            with gr.Group():
                with gr.Row():
                    with gr.Group():
                        recipe_on = gr.Checkbox(label="啟用作者食譜數", value=False)
                        with gr.Row():
                            recipe_min = gr.Number(label="作者食譜數最小值", value=0)
                            recipe_max = gr.Number(label="作者食譜數最大值", value=100)
                    with gr.Group():
                        fans_on = gr.Checkbox(label="啟用作者粉絲數", value=False)
                        with gr.Row():
                            fans_min = gr.Number(label="作者粉絲數最小值", value=0)
                            fans_max = gr.Number(label="作者粉絲數最大值", value=100000)

        search_btn = gr.Button("開始搜尋", elem_id="search-btn")

        search_output = gr.HTML(elem_id="search-output-box")

        search_page_state = gr.State(0)

        search_btn.click(
            fn=dummy_search,
            inputs=[
                text_input, search_number,
                like_on, like_min, like_max,
                together_on, together_min, together_max,
                comment_on, comment_min, comment_max,
                view_on, view_min, view_max,
                recipe_on, recipe_min, recipe_max,
                fans_on, fans_min, fans_max,
            ],
            outputs=[search_output, recipe_html_block, recipes_state, search_page_state]
        )

        with gr.Row():
            search_prev_page_btn = gr.Button("上一頁")
            search_next_page_btn = gr.Button("下一頁")

            search_prev_page_btn.click(
                fn=update_search_page,
                inputs=[search_page_state, recipes_state, gr.State("prev"), text_input, search_number],
                outputs=[search_output, search_page_state]
            )

            search_next_page_btn.click(
                fn=update_search_page,
                inputs=[search_page_state, recipes_state, gr.State("next"), text_input, search_number],
                outputs=[search_output, search_page_state]
            )

    with gr.TabItem("對話機器人"):
        gr.Markdown("## 🍱 請點選你想要分析的食譜（可多選）")
        recipe_page_state = gr.State(0)

        with gr.Row():
            btn_all = gr.Button("全部選擇", elem_id="btn-all", variant="primary")
            btn_none = gr.Button("取消選擇", elem_id="btn-none")

        recipe_html_block.render()

        with gr.Row():
            prev_page_btn = gr.Button("上一頁")
            next_page_btn = gr.Button("下一頁")

            prev_page_btn.click(
                fn=update_recipe_page,
                inputs=[recipe_page_state, recipes_state, gr.State("prev")],
                outputs=[recipe_html_block, recipe_page_state]
            )

            next_page_btn.click(
                fn=update_recipe_page,
                inputs=[recipe_page_state, recipes_state, gr.State("next")],
                outputs=[recipe_html_block, recipe_page_state]
            )

        selected_textbox = gr.Textbox(visible=False, elem_id="selected-textbox")


        with gr.TabItem("是非題"):
        
            true_false_question_input = gr.Textbox(label="輸入你的問題", placeholder="例如：需要巧克力嗎?", value="需要巧克力嗎?")
            true_false_submit_btn = gr.Button("送出")
            true_false_output = gr.HTML(elem_id="true-false-output-box")

            true_false_submit_btn.click(
                fn=handle_true_false_question,
                inputs=[selected_textbox, true_false_question_input, recipes_state],
                outputs=[true_false_output]
            )

        with gr.TabItem("簡答"):
            question_input = gr.Textbox(label="輸入你的問題", placeholder="例如：需要什麼食材?", value="需要什麼食材?")
            submit_btn = gr.Button("送出")
            chat_output = gr.HTML(elem_id="chat-output-box")

            submit_btn.click(
                fn=handle_chat_question,
                inputs=[selected_textbox, question_input, recipes_state],
                outputs=[chat_output]
            )

    with gr.TabItem("檢索食譜"):
        gr.Markdown("### 使用向量檢索快速找到相關食譜")
        with gr.Row():
            rag_question_input = gr.Textbox(label="輸入你的檢索條件", placeholder="例如：用到水果的食譜", value="用到水果的食譜")
            retriever_k_slider = gr.Slider(label="檢索數量 k", minimum=1, maximum=30, value=5, step=1)
        with gr.Row():
            vectorstore_btn = gr.Button("建立並儲存向量資料庫")
            rag_btn = gr.Button("開始檢索", variant="primary")

        vectorstore_state = gr.State()  # 儲存 Chroma vectorstore

        rag_output = gr.HTML(elem_id="rag-output-box")

        vectorstore_btn.click(
            fn=store_vectorstore,
            inputs=[recipes_state],
            outputs=[rag_output, vectorstore_state]
        )

        rag_btn.click(
            fn=handle_rag_question,
            inputs=[rag_question_input, vectorstore_state, retriever_k_slider],
            outputs=[rag_output]
        )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


https://icook.tw/recipes/474792
https://icook.tw/recipes/471502
https://icook.tw/recipes/471112
https://icook.tw/recipes/473628
https://icook.tw/recipes/466969
https://icook.tw/recipes/474367
https://icook.tw/recipes/475359
https://icook.tw/recipes/477266
https://icook.tw/recipes/475854
https://icook.tw/recipes/473930
https://icook.tw/recipes/475387
https://icook.tw/recipes/475303
https://icook.tw/recipes/476946
https://icook.tw/recipes/476164
https://icook.tw/recipes/475106
https://icook.tw/recipes/471223
https://icook.tw/recipes/471742
https://icook.tw/recipes/474623
Processing: https://icook.tw/recipes/474792
Processing: https://icook.tw/recipes/471502
Processing: https://icook.tw/recipes/471112
Processing: https://icook.tw/recipes/473628
Processing: https://icook.tw/recipes/466969
Processing: https://icook.tw/recipes/474367
Processing: https://icook.tw/recipes/475359
Processing: https://icook.tw/recipes/477266
Processing: https://icook.tw/recipes/475854
Processing: https://icook.tw